In [7]:
import logging
import pandas as pd 
import requests
from logging.handlers import RotatingFileHandler
from kafka import KafkaProducer
from kafka.errors import KafkaError
import os
from datetime import date, timedelta
import time
from pandas import json_normalize
import schedule

class StockProducer:
    def __init__(self):
        log_handler = RotatingFileHandler(
            f"./logs/producer.log",
            maxBytes=104857600, backupCount=10)
        logging.basicConfig(
            format='%(asctime)s,%(msecs)d <%(name)s>[%(levelname)s]: %(message)s',
            datefmt='%H:%M:%S',
            level=logging.DEBUG,
            handlers=[log_handler])
        self.logger = logging.getLogger('producer')

        self.producer = KafkaProducer(
            bootstrap_servers=['kafka-1:9092',
                               'kafka-2:9092', 'kafka-3:9092'],
            client_id='producer')

    def message_handler(self, symbol, message):
        #  Message from stock api
        try:
            if not message.empty: 
                stock_info = f"{symbol},{message.Open.iloc[0]},{message.High.iloc[0]},{message.Low.iloc[0]},{message.Close.iloc[0]},{message.Volume.iloc[0]},{message.Tradingdate.iloc[0]}"
                print(symbol + "," + stock_info)
                self.producer.send('stockData', bytes(
                    stock_info, encoding='utf-8'))
                self.producer.flush()
        except KafkaError as e:
            self.logger.error(f"An Kafka error happened: {e}")
        except Exception as e:
            self.logger.error(
                f"An error happened while pushing message to Kafka: {e}")
    def stock_historical_data(self, symbol, start_date, end_date):
        fd = int(time.mktime(time.strptime(start_date, "%Y-%m-%d")))
        td = int(time.mktime(time.strptime(end_date, "%Y-%m-%d")))
        data = requests.get('https://apipubaws.tcbs.com.vn/stock-insight/v1/stock/bars-long-term?ticker={}&type=stock&resolution=D&from={}&to={}'.format(symbol, fd, td)).json()
        df = json_normalize(data['data'])
        df['stockCode'] = symbol
        df.columns = df.columns.str.title()
        return df

    def crawl_from_tcbs(self, symbol_list):
        try:
            self.logger.info("Start running stock producer...")
            for idx, symbol in enumerate(symbol_list):
                start_date = (date.today() - timedelta(days=4)
                              ).strftime("%Y-%m-%d")
                end_date = (date.today()).strftime("%Y-%m-%d")
                
                data = self.stock_historical_data(symbol, start_date, end_date)
                self.message_handler(symbol, data)
            # while True:
            #     pass
        except Exception as e:
            self.logger.error(f"An error happened while streaming: {e}")

    def run(self):
        
        with open("./symbol_list.csv") as f:
            symbol_list = f.read().split('\n')
        # schedule.every().day.at("15:10").do(self.crawl_from_tcbs, c)
        self.crawl_from_tcbs(symbol_list)
        while True:
            schedule.run_pending()
            time.sleep(10)

producer = StockProducer()
producer.run()

            


VVS,VVS,16900.0,16900.0,16000.0,16450.0,200,2024-12-27T00:00:00.000Z
HSV,HSV,4100.0,4100.0,3900.0,3990.0,26523,2024-12-27T00:00:00.000Z
CST,CST,25100.0,25300.0,24800.0,25005.0,60579,2024-12-27T00:00:00.000Z
BVL,BVL,10000.0,10000.0,9300.0,9745.0,8643,2024-12-27T00:00:00.000Z
SGI,SGI,14300.0,14300.0,14300.0,14300.0,0,2024-12-27T00:00:00.000Z
TOS,TOS,74000.0,74200.0,74000.0,74045.0,8932,2024-12-27T00:00:00.000Z
VTZ,VTZ,17300.0,17500.0,17300.0,17400.0,1131900,2024-12-27T00:00:00.000Z
SSH,SSH,67400.0,67500.0,67000.0,67179.0,54700,2024-12-27T00:00:00.000Z
BCA,BCA,12200.0,12200.0,11500.0,11923.0,26900,2024-12-27T00:00:00.000Z
FUCTVGF3,FUCTVGF3,16000.0,16000.0,16000.0,16000.0,100,2024-12-27T00:00:00.000Z
FUEIP100,FUEIP100,8300.0,8350.0,8300.0,8350.0,518,2024-12-27T00:00:00.000Z
GMH,GMH,7800.0,8090.0,7300.0,8090.0,3300,2024-12-27T00:00:00.000Z
BIG,BIG,8100.0,8300.0,6500.0,6973.0,332366,2024-12-27T00:00:00.000Z
CMM,CMM,9200.0,9200.0,9200.0,9200.0,107,2024-12-27T00:00:00.000Z
CNA,CNA,43900.0,4390

KeyboardInterrupt: 

In [2]:
!pip install kafka-python

In [1]:
!pip install --upgrade pip

     |████████████████████████████████| 1.8 MB 30 kB/s             
  Attempting uninstall: pip
    Found existing installation: pip 21.3
    Uninstalling pip-21.3:
      Successfully uninstalled pip-21.3


In [4]:
!pip install schedule